##Importacion de datos


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Instalar las librerias necesarias para la ejecución

!pip install gdown

import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# ID del archivo en Google Drive
file_id = "1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1"
download_url = f"https://drive.google.com/uc?id={file_id}"

# Descargar el archivo CSV a local
output = "archivo.csv"
gdown.download(download_url, output, quiet=False)

# Leer CSV con pandas
df = pd.read_csv(output)



# Importacion del test

file_id = "1c9lxChxn6YhWD9YJCBSdxmA3Xt-fasN7"
download_url = f"https://drive.google.com/uc?id={file_id}"

output = "archivo.csv"
gdown.download(download_url, output, quiet=False)

df_test=pd.read_csv(output)


Downloading...
From (original): https://drive.google.com/uc?id=1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1
From (redirected): https://drive.google.com/uc?id=1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1&confirm=t&uuid=5364d544-af19-4cde-8e0a-e32880c281a2
To: /content/archivo.csv
100%|██████████| 144M/144M [00:01<00:00, 102MB/s]
Downloading...
From: https://drive.google.com/uc?id=1c9lxChxn6YhWD9YJCBSdxmA3Xt-fasN7
To: /content/archivo.csv
100%|██████████| 59.2M/59.2M [00:00<00:00, 77.8MB/s]


El dataframe lo guardamos en la variable df y el que vamos a modificar va a estar en la variable dfa

In [ ]:
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##Lo primero que vamos a hacer es resolver el problema de los datos vacios

In [ ]:
print("El porcentaje de datos vacios en el df es de: ")
print((((df.isna().sum().sum())/(df.shape[0]*df.shape[1]))*(100)),"%")


El porcentaje de datos vacios en el df es de: 
2.0448891181021147 %


Vemos que tenemos demasiados valores vacios, estos representan aproximadamente el 2% del data frame

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import unicodedata
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def preprocess_dataframe(df_input):

    df = df_input.copy()
    # 1. LIMPIEZA INICIAL: Strings 'nan' a NaN real
    df = df.replace(['nan', 'NaN', '', 'None', 'null'], np.nan)

    #
    # 2. ELIMINAR COLUMNAS IDENTIFICADORAS (no aportan)
    cols_to_drop = ['ID', 'PERIODO_ACADEMICO']
    existing_to_drop = [col for col in cols_to_drop if col in df.columns]
    if existing_to_drop:
        df = df.drop(columns=existing_to_drop)


    # 3. IDENTIFICAR TIPOS DE COLUMNAS
    # Guardar columna objetivo si existe
    target_col = 'RENDIMIENTO_GLOBAL'
    has_target = target_col in df.columns

    # Columnas categóricas nominales (para one-hot)
    cols_one_hot = ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO']

    # Columnas ordinales
    cols_ordinales = [
        'E_VALORMATRICULAUNIVERSIDAD',
        'E_HORASSEMANATRABAJA',
        'F_ESTRATOVIVIENDA',
        'F_EDUCACIONPADRE',
        'F_EDUCACIONMADRE'
    ]

    # Columnas binarias
    cols_binarias = [
        'F_TIENEINTERNET',
        'F_TIENELAVADORA',
        'F_TIENEAUTOMOVIL',
        'E_PRIVADO_LIBERTAD',
        'E_PAGOMATRICULAPROPIO',
        'F_TIENECOMPUTADOR',
        'F_TIENEINTERNET_1'
    ]

    # Columnas numéricas
    cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

    # 4. ONE-HOT ENCODING
    cols_to_encode = [col for col in cols_one_hot if col in df.columns]

    if cols_to_encode:
        for col in cols_to_encode:
            # Rellenar NaN con 'Desconocido'
            df[col] = df[col].fillna('Desconocido').astype(str)

        # One-hot encoding
        df = pd.get_dummies(df, columns=cols_to_encode, prefix=cols_to_encode,
                           dtype='int8', drop_first=False)

    # 5. ORDINAL ENCODING

    # 5.1 E_VALORMATRICULAUNIVERSIDAD
    if 'E_VALORMATRICULAUNIVERSIDAD' in df.columns:
        orden_matricula = {
            'No pagó matrícula': 0,
            'Menos de 500 mil': 1,
            'Entre 500 mil y menos de 1 millón': 2,
            'Entre 1 millón y menos de 2.5 millones': 3,
            'Entre 2.5 millones y menos de 4 millones': 4,
            'Entre 4 millones y menos de 5.5 millones': 5,
            'Entre 5.5 millones y menos de 7 millones': 6,
            'Más de 7 millones': 7
        }
        df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].astype(str).map(orden_matricula)
        df['E_VALORMATRICULAUNIVERSIDAD'] = pd.to_numeric(df['E_VALORMATRICULAUNIVERSIDAD'], errors='coerce')
        moda = df['E_VALORMATRICULAUNIVERSIDAD'].mode()[0] if len(df['E_VALORMATRICULAUNIVERSIDAD'].mode()) > 0 else 3
        df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].fillna(moda).astype('int8')

    # 5.2 E_HORASSEMANATRABAJA
    if 'E_HORASSEMANATRABAJA' in df.columns:
        orden_horas = {
            '0': 0,
            'Menos de 10 horas': 1,
            'Entre 11 y 20 horas': 2,
            'Entre 21 y 30 horas': 3,
            'Más de 30 horas': 4
        }
        df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].astype(str).map(orden_horas)
        df['E_HORASSEMANATRABAJA'] = pd.to_numeric(df['E_HORASSEMANATRABAJA'], errors='coerce')
        moda = df['E_HORASSEMANATRABAJA'].mode()[0] if len(df['E_HORASSEMANATRABAJA'].mode()) > 0 else 1
        df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].fillna(moda).astype('int8')

    # 5.3 F_ESTRATOVIVIENDA
    if 'F_ESTRATOVIVIENDA' in df.columns:
        orden_estrato = {
            'Sin Estrato': 0,
            'Estrato 1': 1,
            'Estrato 2': 2,
            'Estrato 3': 3,
            'Estrato 4': 4,
            'Estrato 5': 5,
            'Estrato 6': 6
        }
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].astype(str).map(orden_estrato)
        df['F_ESTRATOVIVIENDA'] = pd.to_numeric(df['F_ESTRATOVIVIENDA'], errors='coerce')
        mediana = df['F_ESTRATOVIVIENDA'].median()
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].fillna(mediana).astype('int8')

    # 5.4 F_EDUCACIONPADRE
    if 'F_EDUCACIONPADRE' in df.columns:
        orden_educacion = {
            'Ninguno': 0,
            'Primaria incompleta': 1,
            'Primaria completa': 2,
            'Secundaria (Bachillerato) incompleta': 3,
            'Secundaria (Bachillerato) completa': 4,
            'Técnica o tecnológica incompleta': 5,
            'Técnica o tecnológica completa': 6,
            'Educación profesional incompleta': 7,
            'Educación profesional completa': 8,
            'Postgrado': 9,
            'No sabe': 0,
            'No Aplica': 0
        }
        df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].astype(str).map(orden_educacion)
        df['F_EDUCACIONPADRE'] = pd.to_numeric(df['F_EDUCACIONPADRE'], errors='coerce')
        mediana = df['F_EDUCACIONPADRE'].median()
        df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].fillna(mediana).astype('int8')

    # 5.5 F_EDUCACIONMADRE
    if 'F_EDUCACIONMADRE' in df.columns:
        orden_educacion = {
            'Ninguno': 0,
            'Primaria incompleta': 1,
            'Primaria completa': 2,
            'Secundaria (Bachillerato) incompleta': 3,
            'Secundaria (Bachillerato) completa': 4,
            'Técnica o tecnológica incompleta': 5,
            'Técnica o tecnológica completa': 6,
            'Educación profesional incompleta': 7,
            'Educación profesional completa': 8,
            'Postgrado': 9,
            'No Aplica': 0
        }
        df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].astype(str).map(orden_educacion)
        df['F_EDUCACIONMADRE'] = pd.to_numeric(df['F_EDUCACIONMADRE'], errors='coerce')
        mediana = df['F_EDUCACIONMADRE'].median()
        df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].fillna(mediana).astype('int8')

    # 6. COLUMNAS BINARIAS (Si/No, S/N)
    for col in cols_binarias:
        if col in df.columns:
            # Convertir todo a string primero
            df[col] = df[col].astype(str).str.strip().str.lower()

            # Mapear a 0/1
            mapeo = {
                'si': 1, 'sí': 1, 's': 1, '1': 1, '1.0': 1, 'true': 1,
                'no': 0, 'n': 0, '0': 0, '0.0': 0, 'false': 0,
                'nan': np.nan, 'none': np.nan
            }
            df[col] = df[col].map(mapeo)
            df[col] = pd.to_numeric(df[col], errors='coerce')

            # Rellenar NaN con moda
            moda = df[col].mode()[0] if len(df[col].mode()) > 0 else 0
            df[col] = df[col].fillna(moda).astype('int8')

    # 7. VARIABLE OBJETIVO (RENDIMIENTO_GLOBAL)
    if has_target:
        orden_rendimiento = {
            'bajo': 0,
            'medio-bajo': 1,
            'medio-alto': 2,
            'alto': 3
        }
        df[target_col] = df[target_col].astype(str).str.lower().str.strip().map(orden_rendimiento)
        df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
        # NO rellenar NaN en el target, dejar como está
        df[target_col] = df[target_col].astype('int8')

    # 8. COLUMNAS NUMÉRICAS (INDICADORES)
    for col in cols_numericas:
        if col in df.columns:
            print(f" Procesando indicador numérico: {col}")
            df[col] = pd.to_numeric(df[col], errors='coerce')
            mediana = df[col].median()
            df[col] = df[col].fillna(mediana).astype('float32')

    # 9. PROCESAR CUALQUIER COLUMNA RESTANTE (por si acaso)
    print(" Verificando columnas no procesadas...")
    for col in df.columns:
        if df[col].dtype == 'object':
            print(f" Columna object encontrada: {col} - Convirtiendo a numérico")
            # Intentar convertir a numérico
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Rellenar NaN con 0
            df[col] = df[col].fillna(0).astype('float32')

    # 10. LIMPIAR NOMBRES DE COLUMNAS (quitar caracteres especiales)
    print("🧹 Limpiando nombres de columnas...")
    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
    df.columns = df.columns.str.strip('_')
    df.columns = df.columns.str.replace('__+', '_', regex=True)  # Reemplazar múltiples _ por uno solo

    # 11. ELIMINAR COLUMNAS DUPLICADAS
    duplicadas_antes = df.columns.duplicated().sum()
    if duplicadas_antes > 0:
        print(f" Encontradas {duplicadas_antes} columnas duplicadas - Eliminando...")
        df = df.loc[:, ~df.columns.duplicated()]

    # ============================================================
    # 12. VERIFICACIÓN FINAL
    # ============================================================
    print("\n" + "="*60)
    print(" PREPROCESAMIENTO COMPLETADO")
    print("="*60)
    print(f" Shape final: {df.shape}")
    print(f" Columnas totales: {len(df.columns)}")
    print(f" Columnas duplicadas: {df.columns.duplicated().sum()}")
    print(f" Tipos de datos:")
    print(df.dtypes.value_counts())

    # Verificar que todo es numérico
    non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        print(f" ADVERTENCIA: Columnas no numéricas encontradas: {non_numeric}")
    else:
        print(" Todas las columnas son numéricas")

    # Verificar NaN
    nan_counts = df.isnull().sum()
    if nan_counts.sum() > 0:
        print(f"\n Valores NaN encontrados en:")
        print(nan_counts[nan_counts > 0])
    else:
        print("No hay valores NaN")

    print("="*60 + "\n")

    return df

In [ ]:
def alinear_columnas(X_train, X_test):
    """Alinea columnas de X_test con X_train"""
    # Agregar columnas faltantes con 0
    missing_cols = set(X_train.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0

    # Eliminar columnas extra
    extra_cols = set(X_test.columns) - set(X_train.columns)
    X_test = X_test.drop(columns=list(extra_cols), errors='ignore')

    # Reordenar para que coincidan
    X_test = X_test[X_train.columns]

    return X_test


In [ ]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

#A partir de este momento queda listo el preprocesado, entonces vamos a comenzar con el primer modelo (Random Forest)

In [ ]:
# Usar la función para preprocesar el DataFrame original
df = preprocess_dataframe(df)


 Procesando indicador numérico: INDICADOR_1
 Procesando indicador numérico: INDICADOR_2
 Procesando indicador numérico: INDICADOR_3
 Procesando indicador numérico: INDICADOR_4
 Verificando columnas no procesadas...
 Columna object encontrada: F_TIENEINTERNET.1 - Convirtiendo a numérico
🧹 Limpiando nombres de columnas...
 Encontradas 67 columnas duplicadas - Eliminando...

 PREPROCESAMIENTO COMPLETADO
 Shape final: (692500, 929)
 Columnas totales: 929
 Columnas duplicadas: 0
 Tipos de datos:
int8       924
float32      5
Name: count, dtype: int64
 Todas las columnas son numéricas
No hay valores NaN



In [ ]:
df.dtypes.unique()

array([dtype('int8'), dtype('float32')], dtype=object)

### Uso de la función de preprocesamiento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
!pip install lightgbm
import lightgbm as lgb


##Hacemos los splits y la distribucion de los datos X y Y

In [ ]:
from sklearn.model_selection import train_test_split

TARGET = "RENDIMIENTO_GLOBAL"

y_train = df[TARGET]
X_train = df.drop(columns=[TARGET])

X_test = preprocess_dataframe(df_test)
X_test = alinear_columnas(X_train, X_test)





 Procesando indicador numérico: INDICADOR_1
 Procesando indicador numérico: INDICADOR_2
 Procesando indicador numérico: INDICADOR_3
 Procesando indicador numérico: INDICADOR_4
 Verificando columnas no procesadas...
 Columna object encontrada: F_TIENEINTERNET.1 - Convirtiendo a numérico
🧹 Limpiando nombres de columnas...
 Encontradas 64 columnas duplicadas - Eliminando...

 PREPROCESAMIENTO COMPLETADO
 Shape final: (296786, 902)
 Columnas totales: 902
 Columnas duplicadas: 0
 Tipos de datos:
int8       897
float32      5
Name: count, dtype: int64
 Todas las columnas son numéricas
No hay valores NaN



In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd

# 1. PREPARAR LOS DATOS
y_train = df[TARGET]
X_train = df.drop(columns=[TARGET])

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train  # Mantener proporción de clases
)



# ============================================================
# 2. CREAR DATASETS DE LIGHTGBM
# ============================================================
train_data = lgb.Dataset(X_train_split, label=y_train_split)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# ============================================================
# 3. PARÁMETROS BÁSICOS
# ============================================================
params = {
    'objective': 'multiclass',
    'num_class': 4,
    'metric': 'multi_logloss',
    'verbose': -1
}

# ============================================================
# 4. ENTRENAR CON EARLY STOPPING
# ============================================================
print("\n Entrenando modelo...")
model = lgb.train(
    params,
    train_data,
    num_boost_round=500,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

print(f"\n Entrenamiento completado!")
print(f" Mejor iteración: {model.best_iteration}")

# ============================================================
# 5. EVALUAR EN VALIDACIÓN
# ============================================================
print("\n Evaluando modelo en validación...")
y_pred_proba = model.predict(X_val, num_iteration=model.best_iteration)
y_pred = np.argmax(y_pred_proba, axis=1)

# Métricas
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_val, y_pred)
print(f"\n Accuracy en validación: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Reporte detallado
class_names = ['Bajo', 'Medio-Bajo', 'Medio-Alto', 'Alto']
print("\n" + "="*60)
print(" REPORTE DE CLASIFICACIÓN")
print("="*60)
print(classification_report(y_val, y_pred, target_names=class_names))

# Matriz de confusión
print("\n" + "="*60)
print(" MATRIZ DE CONFUSIÓN")
print("="*60)
cm = confusion_matrix(y_val, y_pred)
print(cm)

# ============================================================
# 6. GUARDAR MODELO
# ============================================================
model.save_model('lightgbm_model.txt')
print("\n Modelo guardado como 'lightgbm_model.txt'")

# ============================================================
# 7. HACER PREDICCIONES EN TEST (si tienes X_test)
# ============================================================




 Entrenando modelo...
Training until validation scores don't improve for 50 rounds
[50]	train's multi_logloss: 1.22504	valid's multi_logloss: 1.23131
[100]	train's multi_logloss: 1.21031	valid's multi_logloss: 1.21943
[150]	train's multi_logloss: 1.20294	valid's multi_logloss: 1.21517
[200]	train's multi_logloss: 1.1972	valid's multi_logloss: 1.21282
[250]	train's multi_logloss: 1.19256	valid's multi_logloss: 1.21158
[300]	train's multi_logloss: 1.18831	valid's multi_logloss: 1.21067
[350]	train's multi_logloss: 1.18451	valid's multi_logloss: 1.21015
[400]	train's multi_logloss: 1.18084	valid's multi_logloss: 1.20972
[450]	train's multi_logloss: 1.1773	valid's multi_logloss: 1.20935
[500]	train's multi_logloss: 1.17393	valid's multi_logloss: 1.2091
Did not meet early stopping. Best iteration is:
[500]	train's multi_logloss: 1.17393	valid's multi_logloss: 1.2091

 Entrenamiento completado!
 Mejor iteración: 500

 Evaluando modelo en validación...

 Accuracy en validación: 0.4287 (42.87

In [ ]:


print("\n Realizando predicciones en test...")
y_test_pred_proba = model.predict(X_test, num_iteration=model.best_iteration)
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

# Usar el índice de X_test para obtener los IDs correspondientes
class_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
resultados = pd.DataFrame({
    'ID': df_test.loc[X_test.index, 'ID'],  # Usa el índice de X_test
    'RENDIMIENTO_GLOBAL': [class_mapping[p] for p in y_test_pred]
})

print("\n Distribución de predicciones en test:")
print(resultados['RENDIMIENTO_GLOBAL'].value_counts())
print(f"\n Primeras 5 predicciones:")
print(resultados.head())

# Guardar predicciones
resultados.to_csv('predicciones_test.csv', index=False)
print("\n Predicciones guardadas en 'predicciones_test.csv'")


 Realizando predicciones en test...

 Distribución de predicciones en test:
RENDIMIENTO_GLOBAL
bajo          92894
alto          86015
medio-bajo    59572
medio-alto    58305
Name: count, dtype: int64

 Primeras 5 predicciones:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo

 Predicciones guardadas en 'predicciones_test.csv'


In [ ]:
resultados.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
